In [1]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sivasubramanianss40","key":"ffdb5dadbe7d15db6b3aa8a41b97afb7"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle

In [3]:
!kaggle datasets download -d lukechugh/best-alzheimer-mri-dataset-99-accuracy

Dataset URL: https://www.kaggle.com/datasets/lukechugh/best-alzheimer-mri-dataset-99-accuracy
License(s): ODbL-1.0
 95% 68.0M/71.5M [00:03<00:00, 25.7MB/s]
100% 71.5M/71.5M [00:03<00:00, 22.5MB/s]


In [4]:
!unzip best-alzheimer-mri-dataset-99-accuracy.zip -d alzheimers_data

Streaming output truncated to the last 5000 lines.
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1106).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1107).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1108).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1109).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (111).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1110).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1111).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1112).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1113).jpg  
  inflating: alzheimers_data/Combined Dataset/train/No Impairment/NoImpairment (1114).jpg  
  inflating: alzheimers_data/C

In [5]:
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Paths and parameters
train_dir = Path('/content/alzheimers_data/Combined Dataset/train')
img_size, batch_size, epochs = (128, 128), 32, 20


In [6]:
# Function to print the directory structure
def print_directory_structure(path, level=0):
    print('    ' * level + f'[{path.name}]')
    for item in path.iterdir():
        if item.is_dir():
            print_directory_structure(item, level + 1)
        else:
            print('    ' * (level + 1) + item.name)

print("Directory structure:")
print_directory_structure(train_dir)


Streaming output truncated to the last 5000 lines.
        ModerateImpairment (1510).jpg
        ModerateImpairment (2161).jpg
        ModerateImpairment (2543).jpg
        ModerateImpairment (424).jpg
        ModerateImpairment (2036).jpg
        ModerateImpairment (1443).jpg
        ModerateImpairment (1309).jpg
        ModerateImpairment (1526).jpg
        ModerateImpairment (937).jpg
        ModerateImpairment (2317).jpg
        ModerateImpairment (2352).jpg
        ModerateImpairment (2145).jpg
        ModerateImpairment (820).jpg
        ModerateImpairment (606).jpg
        ModerateImpairment (760).jpg
        ModerateImpairment (1304).jpg
        ModerateImpairment (1943).jpg
        ModerateImpairment (380).jpg
        ModerateImpairment (453).jpg
        ModerateImpairment (15).jpg
        ModerateImpairment (2194).jpg
        ModerateImpairment (2313).jpg
        ModerateImpairment (1383).jpg
        ModerateImpairment (1934).jpg
        ModerateImpairment (1881).jpg
        

In [7]:
# Data generators with augmentation
datagen = ImageDataGenerator(
    rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2
)
train_gen = datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training'
)
val_gen = datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation'
)

print(f"Training samples: {train_gen.samples}, Validation samples: {val_gen.samples}")
print(f"Classes: {len(train_gen.class_indices)}, Class indices: {train_gen.class_indices}")


Found 8192 images belonging to 4 classes.
Found 2048 images belonging to 4 classes.
Training samples: 8192, Validation samples: 2048
Classes: 4, Class indices: {'Mild Impairment': 0, 'Moderate Impairment': 1, 'No Impairment': 2, 'Very Mild Impairment': 3}


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

def create_improved_cnn(img_size, num_classes):
    model = models.Sequential([
        # Initial convolution block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_size[0], img_size[1], 3)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.3),

        # Second convolution block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Third convolution block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Fourth convolution block
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Fifth convolution block
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        # Flatten and Dense layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Create and compile the model
img_size = (128, 128)  # Adjust this to match your input image size
num_classes = len(train_gen.class_indices)
model = create_improved_cnn(img_size, num_classes)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 65,120,580 (248.42 MB)

 Trainable params: 65,095,748 (248.32 MB)

 Non-trainable params: 24,832 (97.00 KB)

In [ ]:
# Train the model
history = model.fit(train_gen, epochs=epochs, validation_data=val_gen)
print("Model training completed.")
